# Introduction

Important Info:

Hi and welcome to the Sportlogiq DA Questionnaire! The goal with this questionnaire is to assess your problem solving, hockey knowledge, attention to detail, coding/stats abilities and your (hopeful) fit with many of our day-to-day tasks. Please ensure your answer sheet is completed, but if you find yourself stuck or unsure on a question, showing your work and explaining your problem solving/decision making within your notebook is still highly recommended.

Libraries:
- You'll want to import libraries such as numpy, pandas, shapely, and a plotting library like plotly/matplotlib

Data Info:
- There are two data files & an arena template for plotting. One data file has a condensed event set from a randomly chosen hockey game and the other contains Expected Goals values (xG) to be merged onto shot attempts in the game. **If an xG value does not correspond to a shot event, it should not be counted**

- X and Y Coordinates are in Feet and are adjusted such that both teams attack from left (dz) to right (oz)
    - X values range from -100 (end boards behind the DZ net) to 0 (centre ice) and 100 (end boards behind the OZ net)
    - Y values range from -42.5 (west side boards) to 0 (centre ice) and 42.5 (east side boards)
    - **When creating rink plots, please use these coordinates along with the provided arena_template to display the full rink**

- Binary columns that have values of 0 or 1 indicate 0=No, 1=Yes

- Events or columns that begin with "so" indicate shootout events, "lpr" stands for loose puck recovery

- Successful shots are shots on net, failed shots can be missed or blocked, successful faceoffs are faceoff wins

- Compiledgametime is in seconds, periods are 20 minutes long, except for overtime which is 5 minutes or less

- Video_frame refers to the frame in the video that each event corresponds to. The video uses 30 frames per second for this game

Answer sheet: https://docs.google.com/forms/d/e/1FAIpQLSc4wXZ751nk2IKvipgkrl2g3DJAUOyQ8BrQ1-heCpZsU5EWCg/viewform?usp=sharing&ouid=108145086604888962010

Please show all work, state any assumptions made, **keep long answers succinct and to the point** and most of all, thanks for your time and good luck!

### Q1) Which teamid won the game?

### A1) Team ID 1212 won the game

In [1]:

# import all libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import csv files
main_df = pd.read_csv("DA_tutorial_25.csv")
xg_df = pd.read_csv("DA_tutorial_xG25.csv")

# make new df that shows each teams amount of goals and shootout goals
goal_counts = main_df.groupby("teamid")[["goal", "so_goal"]].sum()
goal_counts["total_goals"] = goal_counts["goal"] + goal_counts["so_goal"]

goal_counts



,goal,so_goal,total_goals
teamid,,,
1212,2,2,4
32522,2,0,2


### Q2) What was the final score?

### A2) 3-2 for Team 1212

From the code above that generates a new df, we can see that team 1212 scored 2 regular goals and 2 shootout goals. Team 32522 scored 2 regular goals and 2 shootout goals meaning that team 1212 won 3-2 in shootout.

### Q3) How many Expected Goals did the shot events for teamid 32522's generate? Please answer rounded to 2 decimal places, ie X.XX

### A3) 3.77, shootout shots did not contribute to the xG

In [3]:
# only inlcude shot events on new df
shot_events = main_df[main_df["eventname"] == "shot"]

# merge both df's
merged_shots = pd.merge(
    shot_events,
    xg_df,
    on=["compiledgametime", "video_frame", "teamid", "playerid"],
    how="left"
)

# only include shot events for team 32522
shots_32522 = merged_shots[
    (merged_shots["teamid"] == 32522) &
    (merged_shots["expectedgoals"].notna())
]

# calc xG for team 32522
xg_sum_32522 = shots_32522["expectedgoals"].sum()
print(f"Total Expected Goals for team 32522: {xg_sum_32522:.2f}")


Total Expected Goals for team 32522: 3.77


### Q4) How many faceoffs did teamid 1212 win? Please answer with a numerical value, not the typed word.

### A4) 25

In [4]:
# filter main_df to successful faceoffs for team 1212, and return number of rows
faceoffs_won_1212 = main_df[
    (main_df["eventname"] == "faceoff") &
    (main_df["outcome"] == "successful") &
    (main_df["teamid"] == 1212)
].shape[0]

faceoffs_won_1212

25

### Q5) How many faceoffs were there in the game? Please answer with a numerical value, not the typed word.

### A5) 66

In [5]:
# find number of rows that event name = faceoff, then divide by 2 since each faceoff is listed twice, once for each team
total_faceoffs = main_df[main_df["eventname"] == "faceoff"].shape[0] // 2

total_faceoffs

66

### Q6) How long was the score differential 2-0 in the game? Please answer with just the numerical value in seconds, rounded to 1 decimal place.

### A6) 49.9

In [6]:
# find just the events of the game that are goals
goal_events = main_df[main_df["goal"] == 1]
goal_events

# now we know that team 1212 went up 2-0 at row 63, then team 32522 scored to make it 2-1 at row 70. now find the difference 
# between these two rows compiledgametimes

time_start = main_df.loc[63, "compiledgametime"]
time_end = main_df.loc[70, "compiledgametime"]

time_end - time_start

49.933340000000044

### Q7) If an Oddman entry is defined as a controlledentryagainst event where there are more attackers than defenders, how many Oddman entries were there in the game? Note: these events have a Type field formatted as Count of Attackers 'on' Count of Defenders, ie 2on3 means 2 attackers and 3 defenders. Please answer with just the numerical value.

### A7) 11

In [7]:
# make a new df that only includes controlled entries
oddman_df = main_df[main_df["eventname"] == "controlledentryagainst"]

# now get values for attackers and defenders from the type column
oddman_df[["attackers", "defenders"]] = oddman_df["type"].str.extract(r"(\d+)on(\d+)").astype(float)

oddman_count = (oddman_df["attackers"] > oddman_df["defenders"]).sum()

oddman_count

<ipython-input-7-970a8e3d77e0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oddman_df[["attackers", "defenders"]] = oddman_df["type"].str.extract(r"(\d+)on(\d+)").astype(float)
<ipython-input-7-970a8e3d77e0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oddman_df[["attackers", "defenders"]] = oddman_df["type"].str.extract(r"(\d+)on(\d+)").astype(float)


11

### Q8) If a "shot off the rush" is defined as the first shot attempt within 5 seconds of an opposing team controlledentryagainst event, with no other events in between, how many shots off the rush did teamid 32522 have? Please answer with just the numerical value.

### A8) 14

In [8]:
# counter variable
rush_shot_count = 0

# iterate thru the entire df
for i in range(len(main_df) - 1):
    curr_row = main_df.iloc[i]
    next_row = main_df.iloc[i + 1]

    # check if current event is controlled entry against team 1212 (which is an entry by team 32522)
    if (curr_row["eventname"] == "controlledentryagainst") and (curr_row["teamid"] == 1212):
        # next event must be a shot, and within 5 seconds
        if (
            (next_row["eventname"] == "shot") and
            (next_row["teamid"] == 32522) and
            (next_row["compiledgametime"] - curr_row["compiledgametime"] <= 5)
        ):
            rush_shot_count += 1

rush_shot_count

14

### Q9) If the slot area includes the innerSlot, eastOuterSlot, and westOuterSlot playsections, what was goalie ID 320320's Slot Save Percentage in the 1st Period? Please answer rounded to 3 decimals with the format X.XXX

### A9) 0.889

In [9]:
# find all events that are shots
shots = main_df[main_df["eventname"] == "shot"]

# now filter for all slot locations
slot_sections = ["innerSlot", "eastOuterSlot", "westOuterSlot"]
shots = shots[shots["playsection"].isin(slot_sections)]

# now just have opposing goalie id 320320
shots = shots[shots["opposingteamgoalieoniceid"] == 320320]

# first period shots, 20 mins = 1200 seconds
shots = shots[shots["compiledgametime"] <= 1200]

# now find saves made and calc save percentage
shots_against = len(shots)
goals_against = shots["goal"].sum()
saves = shots_against - goals_against

save_percentage = saves / shots_against

save_percentage

0.8888888888888888

### Q10) If a rebound shot attempt is defined as two shot events by the same team within 3 seconds of each other, with no other events in between, what was goalie ID 1717716's Goals Saved Above Expected on Rebound Shots? Please answer rounded to 2 decimals, with the format X.XX

### A10) -0.62

In [10]:
# merge df's
merged_df = pd.merge(
    main_df,
    xg_df,
    on=["compiledgametime", "video_frame", "teamid", "playerid"],
    how="left"
)

# sort df by gametime and only include shots
shots = merged_df[merged_df["eventname"] == "shot"].sort_values(by="compiledgametime").reset_index()

# make empty list for the indexs of all shots that are rebounds
rebound_indexes = []

for i in range(len(shots) - 1):
    curr = shots.iloc[i]
    next = shots.iloc[i + 1]

    if (
        curr["teamid"] == next["teamid"] and
        (next["compiledgametime"] - curr["compiledgametime"] <= 3) and
        (next["index"] == curr["index"] + 1) 
    ):
        rebound_indexes.append(next["index"])

# make df of rebound shots
rebound_df = merged_df.loc[rebound_indexes]

# filter df by the right goalie id
rebound_faced = rebound_df[rebound_df["opposingteamgoalieoniceid"] == 1717716]

# calculate goals saved above expected
total_xg = rebound_faced["expectedgoals"].sum()
total_goals = rebound_faced["goal"].sum()
gsax = total_xg - total_goals
gsax

-0.615149646997452

### Q11) Create a plot (with plotly for example) using the arena template provided and add colourful rectangles using the below "shapes" dictionary coordinates for each corresponding colour. If the zone on the right side of the ice is the offensive zone and these rectangles are positions for a powerplay setup, which coloured rectangle best represents the "bumper" spot?
#### Note: for example, the blue rectangle should have points at (50, -37.5), (50, -20), (75, -20), and (75, -37.5)
```
shapes = {
    'blue'  : {'x0':50, 'x1':75, 'y0':-37.5, 'y1':-20}
    ,'green': {'x0':55, 'x1':75, 'y0':10, 'y1':-10}
    ,'red'  : {'x0':50, 'x1':75, 'y0':37.5, 'y1':20}
    ,'purple' : {'x0':27.5, 'x1':45, 'y0':10, 'y1':-10}
    ,'orange' : {'x0':80, 'x1':95, 'y0':-15, 'y1':-25}
    }
```

### A11) The green rectangle best represents the bumper spot on the powerplay, near the slot

In [11]:
# import plotly and image processing library
import plotly.graph_objects as go
from PIL import Image


arena_image_path = "Arena-template.png" 
arena_image = Image.open(arena_image_path)

# define the shapes measurements
shapes = {
    'blue'  : {'x0':50, 'x1':75, 'y0':-37.5, 'y1':-20},
    'green' : {'x0':55, 'x1':75, 'y0':10, 'y1':-10},
    'red'   : {'x0':50, 'x1':75, 'y0':37.5, 'y1':20},
    'purple': {'x0':27.5, 'x1':45, 'y0':10, 'y1':-10},
    'orange': {'x0':80, 'x1':95, 'y0':-15, 'y1':-25}
}

fig = go.Figure()

# add the arena templae as background image
fig.add_layout_image(
    dict(
        source=arena_image,
        xref="x",
        yref="y",
        x=-100,
        y=42.5,
        sizex=200,
        sizey=85,
        sizing="stretch",
        layer="below"
    )
)

#  put in the shapes
for color, coords in shapes.items():
    fig.add_shape(
        type="rect",
        x0=coords["x0"],
        x1=coords["x1"],
        y0=coords["y0"],
        y1=coords["y1"],
        line=dict(color=color),
        fillcolor=color,
        opacity=0.5
    )

fig.update_layout(
    width=800,
    height=400,
    xaxis=dict(range=[-100, 100], scaleanchor="y", showgrid=False, zeroline=False, visible=False),
    yaxis=dict(range=[-42.5, 42.5], showgrid=False, zeroline=False, visible=False),
    title="Powerplay Setup"
)

fig.show()


### Q12) If the "top of the umbrella" corresponds to one of the other coloured rectangles in the "shapes" dictionary above, how many even strength shot attempts took place from within this rectangle? Please answer with just the numerical value.

### A12) 9

In [12]:
# purple regions bounds (top of umbrella)
purple_rect = {'x0': 27.5, 'x1': 45, 'y0': -10, 'y1': 10}

# make new df that filters for just shot attempts at even strength
even_strength_shots = main_df[
    (main_df["eventname"] == "shot") &
    (main_df["manpowersituation"] == "evenStrength")
]

# now filter for just inside purple regions bounds
in_rectangle = even_strength_shots[
    (even_strength_shots["xadjcoord"] >= purple_rect["x0"]) &
    (even_strength_shots["xadjcoord"] <= purple_rect["x1"]) &
    (even_strength_shots["yadjcoord"] >= purple_rect["y0"]) &
    (even_strength_shots["yadjcoord"] <= purple_rect["y1"])
]

# count rows that satisfy these conditions
num_shots = in_rectangle.shape[0]
num_shots


9

### Q13) If the centre of the net is at xadjcoord=89 (goal line), yadjcoord=0 (centre ice), what is the Pearson correlation coefficient between shot attempt distance and expected goals from this game? Please answer with just the numerical value rounded to 2 decimals.

### A13) -0.58

In [13]:
from scipy.stats import pearsonr

# filter for just shots
valid_shots = merged_df[(merged_df["eventname"] == "shot")]

# calc distance to net using euclidean distance
valid_shots["distance"] = np.sqrt((valid_shots["xadjcoord"] - 89)**2 + (valid_shots["yadjcoord"])**2)

# calc coefficient
corr, _ = pearsonr(valid_shots["distance"], valid_shots["expectedgoals"])
corr


<ipython-input-13-28546f3e34ba>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



-0.5770445512429464

### Q14) The coach wants to know if teamid 1212's failed shot attempts (specifically those that were blocked) led to controlled entries against and shot attempts for the opponent. How many of teamid 1212's shots that were blocked led to a controlledentryagainst 1212 followed by a shot attempt for 32522? Hint: assume no time limit exists for the purpose of this exercise. If a faceoff, another shot attempt for 1212, or entry against 32522 take place after the initial blocked shot and before an entry or shot against 1212, do not count these sequences. Please answer with just the numerical value.

### A14) 4

In [14]:
# sort by gametime
main_df_sorted = main_df.sort_values(by="compiledgametime").reset_index(drop=True)

count = 0
for i in range(len(main_df_sorted)):
    row = main_df_sorted.iloc[i]

    # filter for blocked shots from team 1212
    if row["eventname"] == "shot" and row["outcome"] == "failed" and (row["type"] == "outsideblocked" or row["type"] == "slotblocked") and row["teamid"] == 1212:
        blocked_index = i
        valid_sequence = False
        j = blocked_index + 1

        while j < len(main_df_sorted):
            next_event = main_df_sorted.iloc[j]

            # stuff that would make sequence invalid
            if (
                next_event["eventname"] == "faceoff" or
                (next_event["eventname"] == "shot" and next_event["teamid"] == 1212) or
                (next_event["eventname"] == "controlledentryagainst" and next_event["teamid"] == 32522)
            ):
                break

            # correct sequence
            if (
                next_event["eventname"] == "controlledentryagainst" and next_event["teamid"] == 1212
            ):
                # check if next event is sequence
                k = j + 1
                while k < len(main_df_sorted):
                    followup = main_df_sorted.iloc[k]

                    if (
                        followup["eventname"] == "shot" and followup["teamid"] == 32522
                    ):
                        valid_sequence = True
                        break

                    # same stuff for invalid sequence
                    if (
                        followup["eventname"] == "faceoff" or
                        (followup["eventname"] == "shot" and followup["teamid"] == 1212) or
                        (followup["eventname"] == "controlledentryagainst" and followup["teamid"] == 32522)
                    ):
                        break

                    k += 1
                break  

            j += 1

        if valid_sequence:
            count += 1

count


4

### Q15) How many faceoffs at the centre ice faceoff dot did playerid 1346345 win? Please answer with just the numerical value.

### A15) 3

In [15]:
# filter the df to include only faceoffs won by player 1346345 at centre ice
player_faceoffs = main_df_sorted[
    (main_df_sorted["eventname"] == "faceoff") &
    (main_df_sorted["playerid"] == 1346345) & 
    (main_df_sorted["xadjcoord"].between(-1, 1)) & 
    (main_df_sorted["yadjcoord"].between(-1, 1)) & 
    (main_df_sorted["outcome"] == "successful")
]

player_faceoffs.shape[0]

3

### Q16) Make a subset with the data to only include shootout events (these begin with "so") and consider the sequence for each player from "solpr" (puck recovery at centre ice) to "socarry" (crossing the offensive blue line) to "soshot" (shot attempt). What do you notice about each player's shootout attempt? What insights can you suggest for a coach from the data provided? Please answer concisely, but feel free to make a number of clear points.

### A16) Insights:
1. All players took their shot in the shootout from the innerslot play section.
2. PLayers entered the o-zone differently. The players on team 1212 entered at a much wider angle (closer to the boards), while players on team 32522 entered straighter on (closer to centre of ice). Since Team 1212 scored on both attempts and Team 32522 missed both, taking a wider angle in the shootout seams to have a better chance of scoring. This could be due to the goalie having to move laterally to follow players wide angle more and possibly be out of position compared to if the player comes in straight on.
3. The x coordinates of each shot could show whether a player takes a shot from 10-15 feet out, compared to 4-6 feet out which could be a deke. Both teams had the same sequence of: Shooter 1 came in closer to the goalie (deked), then Shooter 2 took a shot from further out. While this is a small sample size, a coach could consider this sequence and potentially have the goalie prepared for a deke vs shot, in this case we would conclude that the lead off shooter is more likely to deke and second shooter will shoot from a bit further out.

In [16]:
shootout_events = main_df[main_df["compiledgametime"] == 3900]
shootout_events

,eventname,outcome,type,compiledgametime,video_frame,playsection,zone,manpowersituation,goal,so_goal,teamid,playerid,opposingteamgoalieoniceid,xadjcoord,yadjcoord
377,solpr,successful,none,3900.0,20304,southCenterNZ,nz,evenStrength,0,0,1212,12789777,9993984,-0.197067,-0.748905
378,socarry,successful,none,3900.0,20368,northEastBoardsNZ,nz,evenStrength,0,0,1212,12789777,9993984,24.949989,28.924622
379,soshot,successful,slot,3900.0,20522,innerSlot,oz,evenStrength,0,1,1212,12789777,9993984,83.291168,-1.754787
380,solpr,successful,none,3900.0,21226,southCenterNZ,nz,evenStrength,0,0,32522,36036,320320,-0.807953,-0.752941
381,socarry,successful,none,3900.0,21269,centerPoint,oz,evenStrength,0,0,32522,36036,320320,25.847931,-9.302940
382,soshot,successful,slot,3900.0,21368,innerSlot,oz,evenStrength,0,0,32522,36036,320320,82.680283,2.264706
383,solpr,successful,none,3900.0,22112,northCenterNZ,nz,evenStrength,0,0,1212,8392384,9993984,0.808815,0.256977
384,socarry,successful,none,3900.0,22176,northWestBoardsNZ,nz,evenStrength,0,0,1212,8392384,9993984,24.949989,-26.398905
385,soshot,successful,slot,3900.0,22271,innerSlot,oz,evenStrength,0,1,1212,8392384,9993984,74.741165,-2.760670
386,solpr,successful,none,3900.0,23156,southCenterNZ,nz,evenStrength,0,0,32522,541541,320320,-0.305008,-0.752941


### Q17) While expected goals values are not provided for the above shootout attempts, if these sequences were actually breakaways during the course of the game, how would you estimate an expected goal value for each "soshot" based on the data provided to you? Please answer concisely, but feel free to make a number of clear points.

### A17) To estimate an xG value for each shootout shot, you could use a regression model that takes in the following features and xG would be the target variable. The features would be: 
1. playerID: this consider which player is shooting and their historical data of scoring.
2. O-zone entry playsection: this will compare to past breakaway attempts, where they entered the zone, and if that ended with a goal.
3. Distance and Angle to net: When taking a shot, taking in the x and y coordinate values can determine how close to the net and at which angle the shot was taken from. This would be compared to past shot attempts taking at similar locations and what the xG for those attempts were.




# Before you submit:

Please check that you have answered all questions to the best of your ability and that every answer can be clearly traced back to work that is shown. If you were unable to produce an answer, please describe and show the work you tried.

Thanks for your time, effort and interest and we hope you enjoyed this exercise!